In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")
companies = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
500


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
#getting url
for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    print(url)


https://www1.salary.com/3m-co-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbvie-inc-Executive-Salaries.html
https://www1.salary.com/abiomed-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
https://www1.salary.com/adobe-inc-Executive-Salaries.html
https://www1.salary.com/advanced-micro-devices-inc-Executive-Salaries.html
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
https://www1.salary.com/aes-corp-Executive-Salaries.html


In [4]:
#Makes a list of to make the graph later on
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

for n in range(len(companies)):
    #getting url
    currName = companies.at[n, 'Security']
    currName1 =companies1.at[n, 'Security']
    col[0][1].append(currName1)
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    page=requests.get(url)
    
    skip = False
    #getting all the tables
    try:
        doc=lh.fromstring(page.content)
        tr_elements=doc.xpath('//tr')
    except:
        skip = True
    
    #indicates that we found the magics on this particular page
    done = True
    
    if skip:
        print(1)
        print(currName1)
        print(currName)
        print(url)
        col[1][1].append('')
        col[2][1].append(0)
        col[3][1].append(0)
        col[4][1].append(0)
    else:
        #this will find the table that has the relevant info for the CEO,and put the correct info into col
        for j in range(0,len(tr_elements)):
            #T is our j'th row
            T=tr_elements[j]

            #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
            #and CEO Pay Ratio)
            if len(T)==4:
                #checks if the current row has the 4 maagic words
                table_Name = ['','','','']
                i=0
                for t in T:
                    name = t.text_content()
                    name = name.__str__()
                    table_Name[i]=name
                    i+=1

                #if it does, I'll take the data from the row right below it
                if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                    #setting done to false
                    done = False
                    T=tr_elements[j+1]
                    #i is the index of our column
                    i=0

                    #this is a temporary storage of the content of each line of the csv
                    content = []

                    #Iterate through each element of the row
                    for t in T.iterchildren():
                        #this part if for cleaning the cell contents
                        data=t.text_content() #gets the content of cell
                        data = data.__str__() #turns content to string
                        data = data.replace(r'$', '') #takes out dollar sign for number conversion
                        data = data.replace(r',', '') #takes out comma for number conversion
                        data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                        data = data.replace(r'CEO Pay', '') #cleans up the content string
                        data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                        data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                        data = data.replace(r'Ratio', '') #cleans up the content string
                        data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                        data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                        content.append(data)

                        #Check if row is empty
                        if i>0:
                        #Convert any numerical value to integers
                            try:
                                data=int(data)
                            except:
                                pass
                        #Append the data to col
                        col[i + 1][1].append(data)
                        #Increment i for the next column
                        i+=1
                    break
        if done:
            print(2)
            print(currName1)
            print(currName)
            print(url)
            col[1][1].append('')
            col[2][1].append(0)
            col[3][1].append(0)
            col[4][1].append(0)

2
Abbott Laboratories
abbott-laboratories
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
2
Accenture plc
accenture-plc
https://www1.salary.com/accenture-plc-Executive-Salaries.html
2
Activision Blizzard
activision-blizzard
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
2
Advance Auto Parts
advance-auto-parts
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
2
AES Corp
aes-corp
https://www1.salary.com/aes-corp-Executive-Salaries.html
2
Alexandria Real Estate Equities
alexandria-real-estate-equities
https://www1.salary.com/alexandria-real-estate-equities-Executive-Salaries.html
2
Alexion Pharmaceuticals
alexion-pharmaceuticals
https://www1.salary.com/alexion-pharmaceuticals-Executive-Salaries.html
2
Align Technology
align-technology
https://www1.salary.com/align-technology-Executive-Salaries.html
2
Allegion
allegion
https://www1.salary.com/allegion-Executive-Salaries.html
2
Amcor plc
amcor-plc
https://www1.salary.com/amcor-plc-Ex

2
Eastman Chemical
eastman-chemical
https://www1.salary.com/eastman-chemical-Executive-Salaries.html
2
Eaton Corporation
eaton-corp
https://www1.salary.com/eaton-corp-Executive-Salaries.html
2
eBay Inc.
ebay-inc
https://www1.salary.com/ebay-inc-Executive-Salaries.html
2
Edison Int'l
edison-intl
https://www1.salary.com/edison-intl-Executive-Salaries.html
2
Edwards Lifesciences
edwards-lifesciences
https://www1.salary.com/edwards-lifesciences-Executive-Salaries.html
2
Electronic Arts
electronic-arts
https://www1.salary.com/electronic-arts-Executive-Salaries.html
2
EOG Resources
eog-resources
https://www1.salary.com/eog-resources-Executive-Salaries.html
2
Equinix
equinix
https://www1.salary.com/equinix-Executive-Salaries.html
2
Estée Lauder Companies
estée-lauder-companies
https://www1.salary.com/estée-lauder-companies-Executive-Salaries.html
2
Etsy
etsy
https://www1.salary.com/etsy-Executive-Salaries.html
2
Evergy
evergy
https://www1.salary.com/evergy-Executive-Salaries.html
2
Expedia Gr

2
McCormick & Co.
mccormick-and-co
https://www1.salary.com/mccormick-and-co-Executive-Salaries.html
2
Mettler Toledo
mettler-toledo
https://www1.salary.com/mettler-toledo-Executive-Salaries.html
2
Microchip Technology
microchip-technology
https://www1.salary.com/microchip-technology-Executive-Salaries.html
2
Micron Technology
micron-technology
https://www1.salary.com/micron-technology-Executive-Salaries.html
2
Mid-America Apartments
mid-america-apartments
https://www1.salary.com/mid-america-apartments-Executive-Salaries.html
2
Mohawk Industries
mohawk-industries
https://www1.salary.com/mohawk-industries-Executive-Salaries.html
2
Molson Coors Beverage Company
molson-coors-beverage-co
https://www1.salary.com/molson-coors-beverage-co-Executive-Salaries.html
2
Mondelez International
mondelez-international
https://www1.salary.com/mondelez-international-Executive-Salaries.html
2
Monster Beverage
monster-beverage
https://www1.salary.com/monster-beverage-Executive-Salaries.html
2
The Mosaic Co

2
Truist Financial
truist-financial
https://www1.salary.com/truist-financial-Executive-Salaries.html
2
Tyler Technologies
tyler-technologies
https://www1.salary.com/tyler-technologies-Executive-Salaries.html
2
Tyson Foods
tyson-foods
https://www1.salary.com/tyson-foods-Executive-Salaries.html
2
Ulta Beauty
ulta-beauty
https://www1.salary.com/ulta-beauty-Executive-Salaries.html
2
U.S. Bancorp
us-bancorp
https://www1.salary.com/us-bancorp-Executive-Salaries.html
2
Under Armour
under-armour
https://www1.salary.com/under-armour-Executive-Salaries.html
2
United Airlines Holdings
united-airlines-holdings
https://www1.salary.com/united-airlines-holdings-Executive-Salaries.html
2
United Parcel Service
united-parcel-service
https://www1.salary.com/united-parcel-service-Executive-Salaries.html
2
Universal Health Services
universal-health-services
https://www1.salary.com/universal-health-services-Executive-Salaries.html
2
Valero Energy
valero-energy
https://www1.salary.com/valero-energy-Executive

In [5]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [6]:
#displaying the dataframe
df

,Company Name,CEO Name,CEO Pay,Median Employee Pay,CEO Pay Ratio
0,3M Company,Michael F. Roman,18321566,57494,319
1,Abbott Laboratories,,0,0,0
2,AbbVie Inc.,Richard A. Gonzalez,21610598,155885,139
3,ABIOMED Inc,Michael R. Minogue,16612766,112393,148
4,Accenture plc,,0,0,0
5,Activision Blizzard,,0,0,0
6,Adobe Inc.,Shantanu Narayen,39145631,147115,266
7,Advanced Micro Devices Inc,Lisa T. Su,58534288,96874,604
8,Advance Auto Parts,,0,0,0
9,AES Corp,,0,0,0


In [7]:
int = 0
for i in range(len(companies)):
    if (df.at[i, 'Median Employee Pay'] == 0):
        int += 1
int

278

In [8]:
int = 0
for i in range(len(companies)):
    if ('News' in df.at[i, 'Company Name']):
        int += 1
        print(i)
int

332


1

In [12]:
df.at[332, 'Company Name']

'News Corp.'

In [13]:
companies.at[332, 'Security']

'news-corp'

In [14]:
companies1.at[332, 'Security']

'News Corp.'

In [16]:
#finding incs

incs = []
int = 0
for i in range(len(companies)):
    if (df.at[i, 'Median Employee Pay'] == 0):
        int += 1
        incs.append(i)
print(int)
incs

278


[1,
 4,
 5,
 8,
 9,
 16,
 17,
 18,
 19,
 25,
 27,
 28,
 31,
 33,
 38,
 39,
 40,
 42,
 44,
 49,
 51,
 53,
 57,
 59,
 62,
 64,
 67,
 69,
 70,
 72,
 73,
 74,
 75,
 77,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 90,
 91,
 92,
 93,
 95,
 96,
 98,
 99,
 101,
 102,
 103,
 105,
 107,
 109,
 110,
 111,
 113,
 115,
 116,
 118,
 119,
 120,
 122,
 124,
 127,
 132,
 133,
 135,
 139,
 140,
 141,
 142,
 144,
 146,
 147,
 148,
 149,
 150,
 154,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 167,
 169,
 172,
 173,
 174,
 178,
 179,
 180,
 182,
 187,
 190,
 193,
 197,
 199,
 200,
 201,
 203,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 214,
 215,
 218,
 220,
 221,
 222,
 223,
 225,
 228,
 229,
 230,
 232,
 233,
 237,
 239,
 243,
 244,
 246,
 247,
 248,
 249,
 250,
 255,
 256,
 257,
 258,
 259,
 260,
 262,
 264,
 268,
 269,
 270,
 271,
 272,
 276,
 277,
 278,
 280,
 281,
 282,
 285,
 291,
 292,
 293,
 294,
 296,
 297,
 298,
 299,
 303,
 310,
 312,
 313,
 315,
 316,
 317,
 318,
 319,
 322,
 328,
 330,
 331,
 332,